In [ ]:
%load_ext autoreload
%autoreload 2
%aimport -raw_data_preprocessing -pandas -numpy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import sys
sys.path.append("../")
sys.path.append("../ukbb_preprocessing/")

from raw_data_preprocessing.raw_data_loader import raw_data_loader
from raw_data_preprocessing.constants import *
from utils import rename_variables, DataRegisterer, percentile

REGISTER = False

loader = raw_data_loader()
registerer = DataRegisterer()

data_asset = loader.ws.data.get(name="clin_ascvd", version="6")
df = pd.read_csv(data_asset.path)
df.index = df.IID.astype(int)

In [ ]:
score = rename_variables(df)
t2d = loader._load_t2d()

score = score.merge(t2d, on='IID', how='left')
score['t2d'] = score['t2d'].astype(bool)
score['t2d_date'] = pd.to_datetime(score['t2d_date'], format='%Y-%m-%d')

score["current_smoker"] = score["smoking"] == 2
score["prior_diabetes"] = score["diabetes"] == 1 | (score["t2d"] & (score["t2d_date"].isna() | (score["t2d_date"] < score["study_date"])))
score["regular_drinker"] = score["alcohol_intake"].isin([1,2])

# from Table 4, https://www.ahajournals.org/doi/suppl/10.1161/01.cir.0000437741.48606.98
# this is the model for white people
pce_m_age                   =  12.344*np.log(score['age'])
pce_m_tc                    =  11.853*np.log(score['tc'])
pce_m_age_tc                = - 2.664*np.log(score['age'])*np.log(score['tc'])
pce_m_hdlc                  = - 7.990*np.log(score['hdlc'])
pce_m_age_hdlc              =   1.769*np.log(score['age'])*np.log(score['hdlc'])
pce_m_current_smoker        =   7.837*score['current_smoker']
pce_m_age_current_smoker    = - 1.795*np.log(score['age'])*score['current_smoker']
pce_m_prior_diabetes        =   0.658*score['prior_diabetes']
pce_m_cvd_meds              = np.where(score['cvd_meds'].str.contains('2', regex=False), 
                                       1.797*np.log(score['systolic_blood_pressure']), 
                                       1.764*np.log(score['systolic_blood_pressure']))

pce_m = 0

for term in [pce_m_age, pce_m_tc, pce_m_age_tc, pce_m_hdlc, pce_m_age_hdlc, pce_m_current_smoker,
             pce_m_age_current_smoker, pce_m_prior_diabetes, pce_m_cvd_meds]:
    term = np.nan_to_num(term, 0)
    pce_m += term

pce_f_age                  = -29.799*np.log(score['age'])
pce_f_tc                   =  13.540*np.log(score['tc'])
pce_f_age_tc               = - 3.114*np.log(score['age'])*np.log(score['tc'])
pce_f_hdlc                 = -13.578*np.log(score['hdlc'])
pce_f_age_hdlc             =   3.149*np.log(score['age'])*np.log(score['hdlc'])
pce_f_current_smoker       =   7.574*score['current_smoker']
pce_f_age_current_smoker   = - 1.665*np.log(score['age'])*score['current_smoker']
pce_f_prior_diabetes       =   0.661*score['prior_diabetes']
pce_f_age2                 =   4.884*np.log(score['age'])**2
pce_f_cvd_meds             = np.where(score['cvd_meds'].str.contains('2', regex=False),
                                      2.019*np.log(score['systolic_blood_pressure']),
                                      1.957*np.log(score['systolic_blood_pressure']))

pce_f = 0

for term in [pce_f_age, pce_f_tc, pce_f_age_tc, pce_f_hdlc, pce_f_age_hdlc, pce_f_current_smoker,
                pce_f_age_current_smoker, pce_f_prior_diabetes, pce_f_age2, pce_f_cvd_meds]:
    term = np.nan_to_num(term, 0)
    pce_f += term

score['y_score'] = np.where(score['sex'] == 1, pce_m, pce_f)

# compute top quantile by sex as threshold
thresholds = score[['sex', 'y_score']].groupby("sex").aggregate(percentile(0.95))
thresholds = thresholds.reset_index()
thresholds.columns = ["sex", "threshold"]

score = score.reset_index().merge(thresholds, on="sex", how="left").set_index('IID')
score["y_pred"] = (score["y_score"] > score["threshold"]).astype(int)

score = score[['y_score', 'y_pred']]

In [ ]:
import os
os.makedirs("pce", exist_ok=True)

splits = loader._load_csv_folder(data_asset_name="ukbb_golden_splits_by_diagnosis_date", version=3)
splits = [split.set_index(split.IID.astype(int)) for split in splits]

for i, s in enumerate(splits):
    s["IID"] = s["IID"].astype(int)
    s_merged = s.set_index("IID").merge(score, on="IID")
    s_merged.to_csv(f'pce/npx_clin_ascvd_{i}_test.csv', sep=',')

In [ ]:
assert merged['pce'].corr(merged['y_score']) > 0.999995

if REGISTER:
    registerer.register_table(data_dir="pce.tsv", name="pce_score", description="PCE score")